In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [3]:
class StudentTranscriptDashboard:
    def __init__(self, dataframe):
        self.df = dataframe
        self.df['Points'] = pd.to_numeric(self.df['Points'], errors='coerce')  # Clean points data
        # Expanded grade mapping to include pass/fail and withdrawals
        self.grade_mapping = {'A+': 4.0, 'A': 4.0, 'A-': 3.7, 'B+': 3.3, 'B': 3.0, 'B-': 2.7, 
                              'C+': 2.3, 'C': 2.0, 'C-': 1.7, 'D+': 1.3, 'D': 1.0, 'D-': 0.7, 'F': 0.0,
                              'P': None, 'NP': None, 'W': None, 'WDN': None}  # 'P' and 'NP' for pass/no pass, 'W' and 'WDN' for withdrawals

    def calculate_gpa(self):
        # Filter out non-numeric grades
        valid_grades_df = self.df[self.df['Grade'].map(self.grade_mapping).notna()]
        valid_grades_df['Weighted Grade'] = valid_grades_df['Units'] * valid_grades_df['Grade'].map(self.grade_mapping)
        total_points = valid_grades_df['Weighted Grade'].sum()
        total_units = valid_grades_df['Units'].sum()
        if total_units > 0:
            gpa = total_points / total_units
        else:
            gpa = 0
        return gpa

    def plot_grade_distribution(self):
        fig = px.histogram(self.df, x='Grade', category_orders={"Grade": [g for g in sorted(self.grade_mapping.keys(), key=lambda x: (self.grade_mapping[x] if self.grade_mapping[x] is not None else -1))]})
        fig.update_layout(title="Grade Distribution")
        return fig

    def plot_course_levels(self):
        self.df['Level'] = self.df['Course Description'].str.extract('(\d+)').astype(int) // 100 * 100
        fig = px.pie(self.df, names='Level', title='Distribution of Course Levels')
        return fig

    def plot_elective_vs_core(self):
        self.df['Course Type'] = self.df['Course Description'].apply(lambda x: 'Core' if 'ELEC' in x or 'MTHE' in x else 'Elective')
        fig = px.box(self.df, x='Course Type', y='Points', title='Elective vs Core Course Performance')
        return fig

    def find_anomalies(self):
        mean = self.df['Points'].mean()
        std_dev = self.df['Points'].std()
        anomalies = self.df[(self.df['Points'] > mean + 2 * std_dev) | (self.df['Points'] < mean - 2 * std_dev)]
        return anomalies

    def flag_failures_and_special_cases(self):
        # Identify failures and non-standard cases
        self.df['Status'] = self.df['Grade'].map(lambda x: 'Fail' if x == 'F' else 'Special' if self.grade_mapping[x] is None else 'Normal')
        flagged = self.df[self.df['Status'] != 'Normal']
        return flagged

if __name__ == "__main__":
    # Usage example
    data = {
        "Course Description": ["APSC 221 Economic And Business Practice", "ELEC 210 Intro Elec Circuits & Machines", "MINE 244 Underground Mining"],
        "Units": [3.00, 4.00, 3.00],
        "Grade": ["A", "WDN", "F"],
        "Points": [12.0, None, 0.0]
    }
    df = pd.DataFrame(data)
    dashboard = StudentTranscriptDashboard(df)
    print("Cumulative GPA:", dashboard.calculate_gpa())
    dashboard.plot_grade_distribution().show()
    dashboard.plot_course_levels().show()
    dashboard.plot_elective_vs_core().show()
    print("Anomalies:\n", dashboard.find_anomalies())
    print("Flagged:\n", dashboard.flag_failures_and_special_cases())

KeyError: 'Points'